# Planet Features API Python Client Introduction


This tutorial is an introduction to [Planet's](https://www.planet.com) Features API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `Planet` module. The Features API is used to create *Collections* made up of *Features* that follow [OGC-compliant features standards](https://ogcapi.ogc.org/features/). The features are stored as GeoJson format FeatureCollections, and will be then be available to use across the Planet APIs or platform, including in Planet Explorer and Features Manager. Your features  are referred to as *items* within the FeatureCollection by the API and API documentation.

## Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's. If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

## Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Features API Documentation](https://docs.planet.com/develop/apis/features/)

The basic workflow for interaction with the Features API is:
1. Define Feature(s) through filepath or text.
2. Create a Collection with a unique name.
3. Add features to the collection.
4. Use feature reference code 'pl:ref' as geometry input in other API calls.

Collections and the Features they hold will be viewable in the Planet Platform [Features Manager](https://www.planet.com/features/). You should be able to see all the Features your org has access to.
<br><br>

Using the workflow in this tutorial will walk you through using a predefined AOI in Cairo, Egypt around Cairo Lake as the feature for our Collection. You may also input your own Feature Collection from an area of your choice.

____

## Set up

In order to interact with the Planet API using the Python client, we need to import the necessary packages and authenticate our Planet account credentials.

### Imports

In [26]:
import geopandas as gpd
import json
import planet
from datetime import datetime
from planet import Auth, Planet, Session, order_request
from planet.geojson import as_ref

### SDK Authentication

Your Planet login is used to authenticate and activate the Python SDK. You will be prompted via a link below to login and confirm on the page that the code displayed matches the authorization code printed. If this is your first time accessing the Planet SDK, you will also be prompted first to authorize the SDK access to your Planet account. 
If you would like to know more, please visit the [authentication documentation](https://docs.planet.com/develop/authentication).

In [4]:
# OAuth2 python client authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = Auth.from_profile('planet-user', save_state_to_storage=False)
auth.ensure_initialized(allow_open_browser=True, allow_tty_prompt=True)

session = Session(auth)
pl = Planet(session)

Opening browser to login.
Confirm the authorization code when prompted: MHGF-KJNC



_____

### Define Features

First, define the FeatureCollection you wish to post to the FeaturesAPI. 

The features must follow the same rules for AOI geometry used in other Planet APIs, including:

- Standard GeoJSON FeatureCollection format.
- Must use WGS84/EPSG:4326 projection.
- Be 2 dimensional.
- Contain 1500 vertices or *less* per feature.

Full feature rules may be found [here](https://docs.planet.com/develop/apis/features/uploading-and-validating-features/#rules-for-creating-a-feature). 
<br> <br>
Below we have defined a FeatureCollection of two areas in Cairo, Egypt. You may use this FeatureCollection to follow along with the tutorial, define your own, or import one from a GeoJSON filepath.

You have a few options to define features to post to your Collections:
- The simplest option is to use a predefined FeatureCollection, so that the Features API can separate each feature into individual items for you.
- You may alternatively pass in just the geometries from feature collections. 
    - There is a free tool [geojson.io](https://geojson.io/) that will allow you to create geometries and FeatureCollections by hand. You can also draw and download custom AOI geojson's from [Planet Explorer](https://www.planet.com/explorer/).

#### Feature Properties

While you only require geometry for Features to be used by the Features API, you may notice that the example FeatureCollection below has additional `properties` in the same way your custom FeatureCollections may also. If a feature is uploaded without specifying the `property_id` parameter (more about this later), all properties of the feature are retained in the item. You can retain extra properties and set your Feature's `id` with an 'id' key. 

**Note** You may optionally give your feature a display name in the Features Manager web interface with a `title` key in the features properties. When you upload a feature, it is not required to have a title. Your feature will still display in the [Features Manager](https://www.planet.com/features/) GUI (displayed as _Untitled Feature_), and function properly in Planet's APIs without a 'Feature Name'.


In [8]:
lake_poly = [[[31.23905453, 29.99579775], [31.26630889, 29.99579775], [31.26630889, 30.01908084], [31.23905453, 30.01908084], [31.23905453, 29.99579775]]]
institute_poly = [[[31.23344423, 30.05109414], [31.25491533, 30.05109414], [31.25491533, 30.06461941], [31.23344423, 30.06461941], [31.23344423, 30.05109414]]]

cairo_fc = {
    "type": "FeatureCollection",
    "features": [
        {
            "id": "0",
            "type": "Feature",
            "properties": {
                "title": "lake_area",
                "name_str": "area_1",
                "extra_property_1": 'extra property 1 here',
                "extra_property_2": 'extra property 2 here'},
            "geometry": {"coordinates": lake_poly, "type": "Polygon"},
        },
        {
            "id": "1",
            "type": "Feature",
            "properties": {
                "title": "institute_area",
                "name_str": "area_2",
                "extra_property_1": 'extra property 1 here',
                "extra_property_2": 'extra property 2 here'},
            "geometry": {"coordinates": institute_poly, "type": "Polygon"},
        },
]}

In [ ]:
# Read your geojson path, then send it to a geo_dict.
geojson_path = 'PATH TO YOUR GEOJSON FILE'

with open(geojson_path, 'r') as file:
    local_fc = json.load(file)

___

### Name and create your FeatureCollection in the Features API

Before you upload your FeatureCollection items, you need to create the Collection using the Features API. If you already have an existing collection and have its id, you may skip creating a new Collection.

Set your `collection_title`, which will serve as the title or name of the whole FeatureCollection. You may optionally set a description for your Collection. `pl.features.create_collection` will create your Collection and return its newly created `collection_id`, which is needed to add items to the Collection.

In [7]:

new_collection_id = pl.features.create_collection(title='cairo_areas', description='FeatureCollection of two areas in Cario, Egypt')
new_collection_id

'cairo_areas-DZ1G8Bq'

### Post Features to your Collection

After the Collection is created, you may post your features from the FeatureCollection we defined above. 
<br> <br>
The `property_id` is optional, it refers to which key from the `properties` block of the supplied feature(s) to use in the `id` for the collection. This gives you some control of the identifier for items without having to change the name of a key to id. Note that an alphanumeric string will be added onto the end of the id even if you set the key to use, to ensure unique ids for all items.
<br> <br>
Without the `property_id` specified, a random string is generated for a features' id, unless the features has a distinct `id` key. 
<br> <br>
The example FeatureColleciton has some extra properties like `name_str` that can be used as the id.

In [ ]:
# Optional
property_id = None

new_items = pl.features.add_items(collection_id=new_collection_id, feature=cairo_fc, property_id=property_id)

In [ ]:
new_items

['pl:features/my/cairo_areas-DZ1G8Bq/0-0a2AVDq',
 'pl:features/my/cairo_areas-DZ1G8Bq/1-mELBv6x']

`add_items` returned a list of strings, which are the Feature Reference IDs of the items we just added to the colleciton. These are used in the `geometry` parameter for Orders and Subscriptions. They are accessible from the `pl:ref` key of an item in the Features API. Before we order imagery with these, we will first explore the other functionalities of the Features python client.

____

### Access your feature Collecitons from the Features API

Basic information (title, description, extent, etc.) about all the feature collections you have access to is available with `list_collections()`

In [13]:
collections_list = list(pl.features.list_collections())
collections_list

[{'id': 'cairo_areas-DZ1G8Bq',
  'title': 'cairo_areas',
  'description': 'FeatureCollection of two areas in Cario, Egypt',
  'item_type': 'feature',
  'created': '2025-12-19T18:43:36Z',
  'updated': '2025-12-19T18:47:47Z',
  'extent': {'spatial': {'bbox': [[31.23344423,
      29.99579775,
      31.26630889,
      30.06461941]]}},
  'links': [{'href': 'https://api.planet.com/features/v1/ogc/my/collections/cairo_areas-DZ1G8Bq',
    'rel': 'self',
    'title': 'This collection',
    'type': 'application/json'},
   {'href': 'https://api.planet.com/features/v1/ogc/my/collections/cairo_areas-DZ1G8Bq/items',
    'rel': 'features',
    'title': 'Features',
    'type': 'application/json'}],
  'feature_count': 2,
  'area': 9890943.929543883,
  'title_property': None,
  'description_property': None,
  'properties': {},
  'permissions': {'can_write': True, 'shared': False, 'is_owner': True},
  'ownership': {'owner_id': 820676, 'org_id': 776979},
  'pl:ref': 'pl:features/my/cairo_areas-DZ1G8Bq'}]

We will use the collection_id of the Collection we created above, `new_collection_id` to list all of its items with `list_items()`:

In [15]:
item_list = list(pl.features.list_items(new_collection_id))
item_list

[{'type': 'Feature',
  'id': '1',
  'properties': {'title': 'institute_area',
   'name_str': 'area_2',
   'extra_property_1': 'extra property 1 here',
   'extra_property_2': 'extra property 2 here',
   'pl:ref': 'pl:features/my/cairo_areas-DZ1G8Bq/1-mELBv6x',
   'pl:area': 3104292.7715780144},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[31.23344423, 30.05109414],
     [31.25491533, 30.05109414],
     [31.25491533, 30.06461941],
     [31.23344423, 30.06461941],
     [31.23344423, 30.05109414]]]}},
 {'type': 'Feature',
  'id': '0',
  'properties': {'title': 'lake_area',
   'name_str': 'area_1',
   'extra_property_1': 'extra property 1 here',
   'extra_property_2': 'extra property 2 here',
   'pl:ref': 'pl:features/my/cairo_areas-DZ1G8Bq/0-0a2AVDq',
   'pl:area': 6786651.157965869},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[31.23905453, 29.99579775],
     [31.26630889, 29.99579775],
     [31.26630889, 30.01908084],
     [31.23905453, 30.01908084],
     [31.23905453

To get a singular item from a feature collection, you need the `collection_id` as well as the `feature_id` itself. We will be reusing the `collection_id` from before, and using the first item in our `items_list`.

In [16]:
feature_id = item_list[0]['id']
pl.features.get_item(new_collection_id, feature_id)

{'type': 'Feature',
 'id': '1',
 'properties': {'title': 'institute_area',
  'name_str': 'area_2',
  'extra_property_1': 'extra property 1 here',
  'extra_property_2': 'extra property 2 here',
  'pl:ref': 'pl:features/my/cairo_areas-DZ1G8Bq/1-mELBv6x',
  'pl:area': 3104292.7715780144},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[31.23344423, 30.05109414],
    [31.25491533, 30.05109414],
    [31.25491533, 30.06461941],
    [31.23344423, 30.06461941],
    [31.23344423, 30.05109414]]]},
 'links': [{'rel': 'self',
   'href': 'https://api.planet.com/features/v1/ogc/my/collections/cairo_areas-DZ1G8Bq/items/1-mELBv6x'},
  {'rel': 'collection',
   'href': 'https://api.planet.com/features/v1/ogc/my/collections/cairo_areas-DZ1G8Bq'}]}

___

### Deleting Collections

Using the `collection_id` you may delete a collection using `delete_collection()`. Be careful when you do this, as there is no confirmation output from this method, and the collection will just be deleted.

In [ ]:
collection_id = ''
pl.features.delete_collection(collection_id = '')

_____

# Using Feature References as input geometries

The Feature Reference IDs, `'pl:ref'` property of a feature is used in other Planet APIs as the geometry parameter instead of having the user track bounding box and geometry variables. Planet `Data`, `Orders`, and `Subscriptions` APIs are all able to make use of a Feature Reference ID. 

PL ref links are submitted in the `geometry` argument in API calls as a dictionary:
```python
geometry = {
    'type': 'ref',
    'content': 'pl:features/my/<collection_id>/<item_id>'
}
```
We can construct this with a helpful function we imported from planet.geojson, `as_ref`, that will return our Feature Reference ID as the dictionary. 

Provided below is consolidated code to use the SDK to get a `pl:ref` from the Features API, use it in a Data API query, and then submit an order request for one item_id clipped to the feature's geometry. If you are unfamiliar with the mechanics of the `order_request` functions of the SDK, the [planet_sdk_orders_demo](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/api_guides/orders_api/planet_sdk_orders_demo.ipynb) notebook will walk you through how to create and submit Orders with the Planet Python SDK.

##### SDK Order workflow:
1. Get Feature Reference ID from Features API and use `as_ref` to format it for use in the variable `geometry_ref`.
2. Run Data API search using `geometry_ref`.
3. Submit order_request to the Orders API using the item_id of the first item returned for our search and `geometry_ref` as the input for our clip tool.

In [ ]:
# List all your collections
collections_list = list(pl.features.list_collections())

# Get the id of the first collection in your list
collection_id = collections_list[0]['id']

collection_items = list(pl.features.list_items(collection_id))
feature_ref = collection_items[0]['properties']['pl:ref']
geometry_ref = as_ref(feature_ref)
geometry_ref

{'type': 'ref', 'content': 'pl:features/my/cairo_areas-DZ1G8Bq/1-mELBv6x'}

To use your Feature Reference ID in Data API, input it as the `geometry` param in `data.create_search`, and not as a geometry filter.

In [ ]:
# Search for PlanetScope ortho_analytic_4b assets within a date range and geometry
name = 'geometry_ref_search'
item_types = ['PSScene']
asset_types = ['ortho_analytic_4b']
start_time = datetime.fromisoformat('2022-05-01T00:00:00Z')
end_time = datetime.fromisoformat('2022-05-02T00:00:00Z')
drf = planet.data_filter.date_range_filter('published', gte=start_time, lte=end_time)

data_filter = planet.data_filter.and_filter([drf])


# Create and run the search:
pl_search = pl.data.create_search(
    item_types=item_types,
    search_filter=data_filter,
    name=name,
    geometry=geometry_ref)

search_results = pl.data.run_search(pl_search['id'])
search_items = list(search_results)

# Get the id of the first item in your search results
item_id = search_items[0]['id']

In [ ]:
item_id

Order request, clipped to the feature:

In [ ]:
product_bundle = 'analytic_udm2'

order_product = order_request.product(
    item_ids=[item_id], product_bundle=product_bundle, item_type = item_types[0])
tools = [order_request.clip_tool(feature_ref)]

In [94]:
name = 'geometry_ref_order'

order_dict = planet.order_request.build_request(name, products = [order_product], tools=tools)

In [95]:
order_dict

{'name': 'geometry_ref_order',
 'products': [{'item_ids': ['20220501_074035_74_2420'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_udm2'}],
 'tools': [{'clip': {'aoi': {'type': 'ref',
     'content': 'pl:features/my/cairo_areas-DZ1G8Bq/1-mELBv6x'}}}]}

In [96]:
order = pl.orders.create_order(request=order_dict)

In [97]:
order

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/1788717c-a458-43f8-8462-6be42fe88970'},
 'created_on': '2025-12-19T22:55:42.249745Z',
 'error_hints': [],
 'id': '1788717c-a458-43f8-8462-6be42fe88970',
 'last_message': 'Preparing order',
 'last_modified': '2025-12-19T22:55:42.249745Z',
 'name': 'geometry_ref_order',
 'products': [{'item_ids': ['20220501_074035_74_2420'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_udm2'}],
 'state': 'queued',
 'tools': [{'clip': {'aoi': {'content': 'pl:features/my/cairo_areas-DZ1G8Bq/1-mELBv6x',
     'type': 'ref'}}}]}